In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import time

# Load your dataset
filename = "NF-BOT-IOT"
train_data = pd.read_csv('/content/drive/MyDrive/CS548 Wireless Project/NF-BOT-IOT_test_preprocessed.csv', sep=',', encoding='utf-8')
test_data = pd.read_csv('/content/drive/MyDrive/CS548 Wireless Project/NF-BOT-IOT_train_preprocessed.csv', sep=',', encoding='utf-8')

X_train = train_data.drop(columns=['label'], axis=1)
y_train = train_data['label']
X_test = test_data.drop(columns=['label'], axis=1)
y_test = test_data['label']

# Define custom K-nearest Neighbor (KNN) algorithm for feature selection
class KNNFeatureSelection:
    def __init__(self, k=5):
        self.k = k

    def fit(self, X, y):
        start_time = time.time()  # Start timing the algorithm
        num_features = X.shape[1]
        best_feature_set = None
        best_accuracy = 0.0

        for feature_idx in range(num_features):
            # Select one feature at a time
            selected_features = [feature_idx]
            clf = KNeighborsClassifier(n_neighbors=self.k)
            X_subset = X.iloc[:, selected_features]
            clf.fit(X_subset, y)
            accuracy = accuracy_score(y, clf.predict(X_subset))

            # Update best solution found
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_feature_set = selected_features

        end_time = time.time()  # End timing the algorithm
        self.exec_time = end_time - start_time  # Calculate execution time
        self.selected_features = best_feature_set
        return self

# Instantiate and run the KNN algorithm for feature selection
knn = KNNFeatureSelection(k=5)
knn.fit(X_train, y_train)

# Apply selected features to training data
selected_feature_index = knn.selected_features[0]
selected_feature = X_train.columns[selected_feature_index]
num_selected_features = 1

# Save selected features to a CSV file
feature_name = filename + "_KNN_features.csv"
optimizer_name = "KNN"
execution_time = knn.exec_time

# Write CSV file with optimizer name, execution time, number of selected features, and their names
with open(feature_name, 'w') as file:
    file.write(f"optimizer,execution time,num of selected features,selected features\n")
    file.write(f"{optimizer_name},{execution_time},{num_selected_features},\"{selected_feature}\"\n")

# Print number of selected features and their names
print("Number of selected features:", num_selected_features)
print("Selected feature:", selected_feature)

# Filter both training and testing data with selected features
X_train_selected = X_train[selected_feature]
X_test_selected = X_test[selected_feature]


Number of selected features: 1
Selected feature: IN_BYTES
